## Preparation

In [1]:
import pandas as pd
import re

**Import Scraped listings**

In [2]:
df_detailed_listings = pd.read_csv("data/eustartup_listings.csv")

df_detailed_listings.head(2)

,name,link,category,based_in,tags,founded,business_name,logo_link,long_business_description,business_description,total_funding,website,company_status,social_links
0,Avdain,https://www.eu-startups.com/directory/avdain/,Austria,Vienna,"Company, Startup, One Person",2020.0,Avdain,https://www.eu-startups.com/wp-content/uploads...,Avdain is a enterprise that embodies a fusion ...,Avdain is an technology company founded and so...,No funding announced yet,avdain.com,Active,()
1,PDF To Brainrot,https://www.eu-startups.com/directory/pdf-to-b...,Austria,"245 Wo Lung Street, Fanling, North District, H...","AI,Video,Learning",2024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,()


## Preprocess Listings

In [3]:
nan_rows = df_detailed_listings[df_detailed_listings.isnull().any(axis=1)]
nan_rows.head()

,name,link,category,based_in,tags,founded,business_name,logo_link,long_business_description,business_description,total_funding,website,company_status,social_links
1,PDF To Brainrot,https://www.eu-startups.com/directory/pdf-to-b...,Austria,"245 Wo Lung Street, Fanling, North District, H...","AI,Video,Learning",2024.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,()
3,Popper Power GmbH,https://www.eu-startups.com/directory/popper-p...,Austria,Vienna,"EV, Battery, BESS, Charging",2022.0,Popper Power GmbH,https://www.eu-startups.com/wp-content/uploads...,NaN,Popper Power GmbH develops advanced energy sto...,Between €500K-€ 1 million,www.popperpower.com,Active,"('https://www.linkedin.com/company/86313916',)"
9,Pressure Washing Albany,https://www.eu-startups.com/directory/pressure...,Austria,loudonville,Pressure Washing,2024.0,Pressure Washing Albany,NaN,Pressure Washing Albany 4 Beaver Pond Rd Loudo...,Pressure Washing Albany 4 Beaver Pond Rd Loudo...,No funding announced yet,https://pressurewashingalbanyny.com/,NaN,()
10,lxbfYeaa,https://www.eu-startups.com/directory/no-title/,Austria,lxbfYeaa,1,2024.0,(no title),NaN,NaN,NaN,NaN,NaN,NaN,()
16,Finslice,https://www.eu-startups.com/directory/finslice/,Austria,Linz,"Fintech, Exchange, Alternative Investments, Ta...",2024.0,Finslice,https://www.eu-startups.com/wp-content/uploads...,NaN,Finslice is an exchange that enables users to ...,No funding announced yet,https://www.finslice.eu/,Active,('https://www.linkedin.com/company/finslice-eu...


### Top 10 Values for each column

In [4]:
# show *all* rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# widen the “page” so it won’t wrap columns  
pd.set_option('display.width', 1000)

In [5]:
top10_data = {}

for col in df_detailed_listings.columns:
    # include NaNs, then take the top 10
    vc = df_detailed_listings[col] \
         .value_counts(dropna=False) \
         .head(10)

    # (optional) relabel the NaN index to "<Missing>"
    vc.index = vc.index.where(~vc.index.isna(), '<Missing>')

    top_values = vc.index.tolist()
    top_counts = vc.values.tolist()

    top10_data[col]       = pd.Series(top_values)
    top10_data[f"{col}_count"] = pd.Series(top_counts)

top10_df = pd.DataFrame(top10_data)

display(top10_df)

,name,name_count,link,link_count,category,category_count,based_in,based_in_count,tags,tags_count,founded,founded_count,business_name,business_name_count,logo_link,logo_link_count,long_business_description,long_business_description_count,business_description,business_description_count,total_funding,total_funding_count,website,website_count,company_status,company_status_count,social_links,social_links_count
0,Storyblok,3,https://www.eu-startups.com/directory/matchoff...,1,Austria,700.0,Vienna,400,BigData,6,2019.0,135,Storyblok,3,<Missing>,4,<Missing>,575,<Missing>,3,No funding announced yet,344,https://www.digando.com,3,<Missing>,439.0,(),486
1,506.ai,2,https://www.eu-startups.com/directory/avdain/,1,"Austria,Germany",2.0,Graz,55,<Missing>,5,2020.0,134,506.ai,2,https://www.eu-startups.com/wp-content/uploads...,1,Avdain is a enterprise that embodies a fusion ...,1,DrainBot provides railway and road operators w...,2,<Missing>,122,<Missing>,2,Active,259.0,"('https://www.linkedin.com/company/86313916',)",1
2,WeDress Collective,2,https://www.eu-startups.com/directory/pdf-to-b...,1,NaN,NaN,Linz,54,"Artificial Intelligence, Information Technolog...",2,2021.0,89,WeDress Collective,2,https://www.eu-startups.com/wp-content/uploads...,1,"Beyond a starter: Your complete project, ready...",1,Coworking Wien - Shared Office Wien,1,Between €100K-€500K,66,https://medcom.at/,2,Idea Stage,4.0,('https://www.linkedin.com/company/SurveySensu...,1
3,blackshark.ai,2,https://www.eu-startups.com/directory/softgen/,1,NaN,NaN,Innsbruck,16,Health,2,2022.0,54,blackshark.ai,2,https://www.eu-startups.com/wp-content/uploads...,1,Speed matters when you want more sales. That’s...,1,Avdain is an technology company founded and so...,1,Between €1 million-€ 2.5 million,47,https://www.roi4presenter.com/,2,NaN,NaN,('https://www.linkedin.com/in/sherin-shakirem-...,1
4,S-A-M Technologies,2,https://www.eu-startups.com/directory/popper-p...,1,NaN,NaN,Wien,13,"Finance, Financial Services, Real Estate",2,2017.0,53,S-A-M Technologies,2,https://www.eu-startups.com/wp-content/uploads...,1,SurveySensum is a leading customer feedback pl...,1,Softgen is your AI Web App Developer. Describe...,1,Between €500K-€ 1 million,29,www.campstar.com,2,NaN,NaN,"('https://www.linkedin.com/in/gerhardkuerner/',)",1
5,MedCom,2,https://www.eu-startups.com/directory/setter-ai/,1,NaN,NaN,Salzburg,9,"Biotechnology, Pharmaceutical",2,2018.0,46,MedCom,2,https://www.eu-startups.com/wp-content/uploads...,1,Traditional AI workflows often involve navigat...,1,Popper Power GmbH develops advanced energy sto...,1,Between €2.5 million-5 million,29,www.kompany.com,2,NaN,NaN,('https://www.linkedin.com/company/finslice-eu...,1
6,Campstar,2,https://www.eu-startups.com/directory/surveyse...,1,NaN,NaN,Klagenfurt,8,Sustainability,2,2023.0,41,Campstar,2,https://www.eu-startups.com/wp-content/uploads...,1,Share Your Party is a dynamic platform designe...,1,Easy-to-use AI appointment setter for WhatsApp...,1,Between €1-€100K,27,https://www.prop.id/,2,NaN,NaN,"('https://www.linkedin.com/company/daiki-ai/',)",1
7,DrainBot,2,https://www.eu-startups.com/directory/artypa/,1,NaN,NaN,Dornbirn,5,"Information Services, Information Technology, ...",2,2015.0,37,DrainBot,2,https://www.eu-startups.com/wp-content/uploads...,1,PodPally – Your Podcast’s New Best Friend Say ...,1,SurveySensum is a customer feedback platform t...,1,Above €25 million,16,https://www.wedresscollective.com/home,2,NaN,NaN,('https://www.linkedin.com/company/98882864/ad...,1
8,kontractory,2,https://www.eu-startups.com/directory/share-yo...,1,NaN,NaN,Klosterneuburg,4,"AI, imaging diagnostic, digital health",2,2016.0,34,kontractory,2,https://www.eu-startups.com/wp-content/uploads...,1,Pressure Washing Albany 4 Beaver Pond Rd Loudo...,1,Artypa addresses the limitations of fragmented...,1,Between €5 million-€10 million,16,https://kontractory.com,2,NaN,NaN,('https://www.linkedin.com/company/19018800/ad...,1
9,Digando.com,2,https://www.eu-startups.com/directory/podpa

In [6]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

### Remove duplicates based on "website" and "name"

In [7]:
# Drop duplicates based on website
df_detailed_listings.drop_duplicates(
    subset=['website'],
    keep='first',
    inplace=True
)

In [8]:
# Drop duplicates based on "name"
df_detailed_listings.drop_duplicates(subset=['name'], keep='first', inplace=True)

### Process long_description

In [9]:
# Overwrite business_description with long_business_description where available
df_detailed_listings['business_description'] = (
    df_detailed_listings['long_business_description']
    .combine_first(df_detailed_listings['business_description'])
)

# Drop the now-unneeded long_business_description column
df_detailed_listings.drop(columns=['long_business_description'], inplace=True)

### Drop Missing values

In [10]:
print("Before:", len(df_detailed_listings), "rows")

cols_to_keep = [
    'name',
    'based_in',
    'tags',
    'founded',
    'logo_link',
    'business_description',
    'total_funding',
    'website'
]

df_detailed_listings.dropna(
    subset=cols_to_keep,
    how='any',
    inplace=True
)

print("After: ", len(df_detailed_listings),           "rows")

Before: 673 rows
After:  553 rows


### Batch into regions

In [11]:
# Normalize: strip whitespace and lowercase everything
df_detailed_listings['based_in_norm'] = df_detailed_listings['based_in'].astype(str).str.strip().str.lower()

# Regex mapping dict
region_map = {
  # Vienna
  r'^(vienna|wien|viena|vienna, austria|vienna / zürich|vienna, london|vienna, wien|atzgersdorf)(/.*)?$':'Vienna',
  # Vorarlberg
  r'.*(vorarlberg|lochau|lustenau|dornbirn|bregenz|wolfurt).*':'Vorarlberg',
  # Burgenland
  r'.*(burgenland|mattersburg|eisenstadt|piringsdorf|parndorf|pinkafeld).*':'Burgenland',
  # Carinthia
  r'.*(carinthia|kärnten|spittal an der drau|klagenfurt|krumpendorf|moosburg).*':'Carinthia',
  # Lower Austria
  r'.*(lower austria|niederosterreich|nieder-?österreich|ober-wolfsbach|euratsfeld|traismauer|tulln|kritzendorf|klosterneuburg|ternitz|gerasdorf bei wien|bad voslau|herzogenburg|wiener neustadt|bruck an der leitha|wiener neudorf|brunn am gebirge|neidling|rohrau|wieselburg|kirchbach|biedermannsdorf|wolkersdorf im weinviertel|bisamberg|wöllersdorf).*':'Lower Austria',
  # Upper Austria
  r'.*(upper austria|oberosterreich|ober-?österreich|linz|selker|eglau|st. martin im mühlkreis|giga|wolfern|wels|sankt georgen an der gusen|hagenberg|leonding|schörfling|perg|tiefgraben|braunau am inn|micheldorf|sierning|weibern|lenzing|bad goisern|steyr|gallspach).*':'Upper Austria',
  # Salzburg
  r'.*(salzburg|oberalm|hallein|saalfelden).*':'Salzburg',
  # Styria
  r'.*(styria|steiermark|graz|thal|fernitz-mellach|voitsberg|premstatten|gleisdorf|leibnitz|leoben|raaba|giessenberg|seiersberg-pirka|aschbach).*':'Styria',
  # Tyrol
  r'.*(tyrol|tirol|innsbruck|wattens|sankt johann|kufstein|zams|jenbach|gasse).*':'Tyrol',
  # Other
  r'.*(austria|österreich|collyer quay|ahmedabad|lxbfyeaa|loudonville|tempe|245 wo lung street, fanling, north district, hong kong|redfern|berkeley|boorklyn|baia mare|pakistan|gulbarga|software).*':'Other'
}

# Regex replace
df_detailed_listings['based_in_norm'] = df_detailed_listings['based_in_norm'].replace(region_map, regex=True)

print(df_detailed_listings['based_in_norm'].value_counts())

based_in_norm
Vienna           335
Upper Austria     71
Styria            52
Lower Austria     29
Tyrol             24
Salzburg          13
Other              9
Vorarlberg         9
Carinthia          7
Burgenland         4
Name: count, dtype: int64


In [12]:
# check for missing values
df_detailed_listings[df_detailed_listings['based_in_norm'].isnull()]

,name,link,category,based_in,tags,founded,business_name,logo_link,business_description,total_funding,website,company_status,social_links,based_in_norm


**Remove others (listings not out of Austria)**

In [13]:
valid_regions = [
    'Vienna', 'Vorarlberg', 'Burgenland', 'Carinthia',
    'Lower Austria', 'Upper Austria', 'Salzburg',
    'Styria', 'Tyrol'
]

df_detailed_listings = df_detailed_listings[
    df_detailed_listings['based_in_norm'].isin(valid_regions)
].copy()

print(df_detailed_listings['based_in_norm'].value_counts())

based_in_norm
Vienna           335
Upper Austria     71
Styria            52
Lower Austria     29
Tyrol             24
Salzburg          13
Vorarlberg         9
Carinthia          7
Burgenland         4
Name: count, dtype: int64


### Batch Tags into categories

It cleans and lowercases each startup’s tags, then uses regex patterns to map those tags to sectors. Finally, it assigns each company the most frequent sector (with name-based or “Other” fallbacks) and stores that in a new primary_category column.

In [14]:
df_detailed_listings.tags.value_counts()

tags
BigData                                                                                                       6
Sustainability                                                                                                2
Artificial Intelligence, Information Technology, Machine Learning, Product Search, Software, Visual Search    2
Biotechnology, Pharmaceutical                                                                                 2
Finance, Financial Services, Real Estate                                                                      2
                                                                                                             ..
Pet tech, GPS & Health Tracker                                                                                1
B2B ,FinTech, GovTech, Information Services, Information Technology                                           1
Platform, Translation, Translation Service, Enterprise Software, Localization                      

In [15]:
import re
from collections import Counter
import pandas as pd

# 1. Normalize the raw 'tags' column into a clean, lowercase, whitespace-trimmed string
df = df_detailed_listings
df['tags_norm'] = (
    df['tags']
      .fillna('')
      .astype(str)
      .str.strip()
      .str.lower()
)

# 2. Define a regex→sector mapping dict
tags_map = {
  # Mobility
  r'.*(^mobility$|logistics|transportation|delivery service|automotive|taxi|supply chain|boat rental|passenger|navigation|aerospace|drivers via|campervan rental).*':'Mobility',
  # Health
  r'.*(^health$|health,|fitness|medtech|biotechnology research|healthcare|pharma|^sports$|^sports,|, sports|medical|hospitals|health care|wellbeing|biotechnology|nursing|therapy|healthtech|health diagnostics|sexual passions).*':'Health',
  # Education
  r'.*(education|, edtech|^edtech$|^edtech,|^training,|, training,|,training,|e-learning|onlinelearning|tutoring).*':'Education',
  # Software & Analytics
  r'.*(bigdata|machine learning|analytics|information|behaviour|behavior|monitoring|data|event|marketing|network security|internet, platforms|website compliance|java, spring boot|video blurring|video, conference|document processing|business advisor|incident response|contract management|vibration engineering|one person|crm|health diagnostics|echo, audio|ai,video,learning|developing tool|augmented reality|ai code assistant|tracking|^ai$|^ai,|, ai|,ai| ai,|ai,|ai.|cyber security|ai productivity|customer feedback|handwritten text recognition|research|cloud computing|software|saas|mobile app|apps|artificial intelligence|automation|gps|anomaly detection|^plot| plot|seo|web development|database| iot|^iot|pdf|productivity|map|editing|virtual reality|elderly, demographic change|safety app).*':'Software & Analytics',
  # Hardware
  r'.*(hardware|pressure washing|bess|sensor|electronics|manufacturing|electroporation|micro speakers|biometric|aerospace|semiconductor|dive, scuba|industrial, infrastructure|vibration engineering).*':'Hardware',
  # ClimateTech/GreenTech/CleanTech
  r'.*(sustainability|sustainable|waste management|recycling|paperless|food rescue|hydroponic|glacier|biodegradable|fashion, internet).*':'ClimateTech/GreenTech/CleanTech',
  # Energy
  r'.*(energy|battery).*':'Energy',
  # e-commerce
  r'.*(e-commerce|customer feedback|retail|outfits|marketing|ecommerce|fashion, internet|buy and sell second-hand).*':'e-commerce',
  # PropTech
  r'.*(proptech|office rentals|coworking|real estate|hospitality|hotel|travel tech|traveltech|smart home|booking, travel|workations).*':'PropTech',
  # AgTech/FoodTech
  r'.*(^agtech$|foodtech|food|farmtech|agritech|^farming$|horse|water purification|healthiest drink|harvest estimation|brewery|microorganism, gas fermentation).*':'AgTech/FoodTech',
  # FinTech/InsurTech
  r'.*(finance|insurance|fintech|blockchain|marketplace|fundraising|expense, savings|payment|cloudexit|tax|invest|cash-flow).*':'FinTech/InsurTech',
  # Professional Services
  r'.*(services|consulting|catering|business verification|crafts|tutoring|nursing|therapy|incident response|online rental for construction equipment|young, modern, nice, professional|legal-tech|business advisor|industrial, infrastructure|white-label metaverse solution|business agility|legal department|digital agency|it, developers, businesses|coaching|booking|webdesign|older generation|travel agency|campervan rental|tourists, travel, b2b).*':'Professional Services',
  # Media & Entertainment
  r'.*(media|entertainment|podcast preparation|news|party|wedding|content|video games|culttech|spectator sports|audio, social, app|live streaming service|video, conference|social network|gaming|visitors|spiel|smashcast|game development|computer game|video streaming|sexual passions|govtech|video, freemium).*':'Media & Entertainment',
  # Recruitment
  r'.*(recruitment|companies rating|human resources|recruiting|hr tech|staff|scouting|job|elderly, demographic change|dev-employerbranding).*':'Recruitment',
  # ConstructionTech/Green Building
  r'.*(constructiontech|green building|^construction,|^construction$|, construction,|, house|, house,|,house,|online rental for construction equipment).*':'ConstructionTech/Green Building'
}

# 3. Pre-compile all regex patterns once for efficiency
compiled_patterns = [(re.compile(pattern), sector) for pattern, sector in tags_map.items()]

# 4. Function to map an individual tag fragment to its sector (returns None if no match)
def map_tag_to_category(tag_fragment):
    for regex, sector in compiled_patterns:
        if regex.search(tag_fragment):
            return sector
    return None

# 5. Define fallback logic for companies with no valid tag matches
no_tags_map = {
    r'(YepOffers Austria|Userbrain|flight4sale\.com)': 'Software & Analytics',
    r'refurbed\.de': 'ClimateTech/GreenTech/CleanTech,e-commerce',
    r'riskine': 'FinTech/InsurTech'
}

# 6. For each row, split tags, map each to a sector, filter out None, then choose mode
def derive_primary_category(row):
    tag_str, name = row['tags_norm'], row['name']
    # Split the comma-separated tags into clean tokens
    tokens = [tok.strip() for tok in tag_str.split(',') if tok.strip()]
    # Map tokens to sectors, dropping any that didn’t match
    matched = [map_tag_to_category(tok) for tok in tokens]
    valid = [m for m in matched if m is not None]
    if valid:
        # Return the sector that appears most often
        return Counter(valid).most_common(1)[0][0]
    # If no tags matched, try the company name fallback
    for pattern, sector in no_tags_map.items():
        if re.search(pattern, name, flags=re.IGNORECASE):
            return sector
    # Ultimate default
    return 'Other'

# 7. Apply the primary-category function row-wise
df['primary_category'] = df.apply(derive_primary_category, axis=1)

# 8. Display the final counts for check
print(df['primary_category'].value_counts())

primary_category
Software & Analytics               274
Health                              55
Professional Services               55
FinTech/InsurTech                   31
Media & Entertainment               20
AgTech/FoodTech                     15
Hardware                            13
Education                           13
Mobility                            13
Other                               12
PropTech                            11
Energy                              10
e-commerce                          10
ConstructionTech/Green Building      6
Recruitment                          4
ClimateTech/GreenTech/CleanTech      2
Name: count, dtype: int64


### Age column

In [16]:
# Drop any rows where 'founded' is missing
df_detailed_listings.dropna(subset=['founded'], inplace=True)

# Convert 'founded' to integer years
df_detailed_listings['founded'] = df_detailed_listings['founded'].astype(int)

# Compute age as of 2025
df_detailed_listings['age'] = 2025 - df_detailed_listings['founded']

# Drop all rows where 'founded' is outside [2015, 2024]
out_of_range = (
    (df_detailed_listings['founded'] < 2015) |
    (df_detailed_listings['founded'] > 2024)
)
df_detailed_listings.drop(index=df_detailed_listings[out_of_range].index, inplace=True)

print(df_detailed_listings[['name','founded','age']].head())

                name  founded  age
0             Avdain     2020    5
3  Popper Power GmbH     2022    3
4          Setter AI     2024    1
5       SurveySensum     2018    7
6             Artypa     2024    1


In [17]:
print(df_detailed_listings['founded'].value_counts())

founded
2020    123
2019    116
2021     87
2022     52
2023     38
2018     31
2017     24
2024     21
2015     15
2016     15
Name: count, dtype: int64


### Company Status

drop idea stage

In [18]:
df_detailed_listings = df_detailed_listings[
    df_detailed_listings['company_status'] != 'Idea Stage'
].copy()

## Check Final results and save

#### Reorder and drop columns

In [22]:
columns_list = df_detailed_listings.columns.tolist()

for column in columns_list: 
    print(column)

name
link
category
based_in
tags
founded
business_name
logo_link
business_description
total_funding
website
company_status
social_links
based_in_norm
tags_norm
primary_category
age
total_funding_norm


In [23]:
# define the exact column order
keep_cols = [
    'name',
    'link',
    'logo_link',
    'website',
    'based_in',
    'based_in_norm',
    'tags_norm',
    'primary_category',
    'business_description',
    'founded',
    'age',
    'total_funding',
    'company_status',
    'social_links'
]

# subset & reorder in-place (with a copy to avoid SettingWithCopyWarning)
df_detailed_listings = df_detailed_listings[keep_cols].copy()

In [24]:
df_detailed_listings.rename(columns={
    'link'             : 'link_startupeu',
    'logo_link'        : 'link_logo',
    'based_in'         : 'city',
    'based_in_norm'    : 'region',
    'tags_norm'        : 'tags',
    'primary_category' : 'category'
}, inplace=True)

#### Check results

In [25]:
# show *all* rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# widen the “page” so it won’t wrap columns  
pd.set_option('display.width', 1000)

In [26]:
top10_data = {}

for col in df_detailed_listings.columns:
    # include NaNs, then take the top 10
    vc = df_detailed_listings[col] \
         .value_counts(dropna=False) \
         .head(10)

    # (optional) relabel the NaN index to "<Missing>"
    vc.index = vc.index.where(~vc.index.isna(), '<Missing>')

    top_values = vc.index.tolist()
    top_counts = vc.values.tolist()

    top10_data[col]       = pd.Series(top_values)
    top10_data[f"{col}_count"] = pd.Series(top_counts)

top10_df = pd.DataFrame(top10_data)

display(top10_df)

,name,name_count,link_startupeu,link_startupeu_count,link_logo,link_logo_count,website,website_count,city,city_count,region,region_count,tags,tags_count,category,category_count,business_description,business_description_count,founded,founded_count,age,age_count,total_funding,total_funding_count,company_status,company_status_count,social_links,social_links_count
0,Vira Therapeutics,1,https://www.eu-startups.com/directory/vira-the...,1,https://www.eu-startups.com/wp-content/uploads...,1,https://www.viratherapeutics.com/,1,Vienna,298,Vienna,316.0,bigdata,6,Software & Analytics,259,ViraTherapeutics develops a new class of immun...,1,2020,123,5,123,No funding announced yet,305.0,<Missing>,274.0,(),313
1,Avdain,1,https://www.eu-startups.com/directory/avdain/,1,https://www.eu-startups.com/wp-content/uploads...,1,avdain.com,1,Linz,44,Upper Austria,69.0,sustainability,2,Professional Services,55,Avdain is a enterprise that embodies a fusion ...,1,2019,116,6,116,Between €100K-€500K,64.0,Active,244.0,"('https://www.linkedin.com/company/86313916',)",1
2,Popper Power GmbH,1,https://www.eu-startups.com/directory/popper-p...,1,https://www.eu-startups.com/wp-content/uploads...,1,www.popperpower.com,1,Graz,39,Styria,48.0,"artificial intelligence, information technolog...",2,Health,53,Popper Power GmbH develops advanced energy sto...,1,2021,87,4,87,Between €1 million-€ 2.5 million,44.0,NaN,NaN,('https://www.linkedin.com/company/SurveySensu...,1
3,Setter AI,1,https://www.eu-startups.com/directory/setter-ai/,1,https://www.eu-startups.com/wp-content/uploads...,1,https://www.trysetter.com,1,Innsbruck,13,Lower Austria,29.0,"biotechnology, pharmaceutical",2,FinTech/InsurTech,30,Speed matters when you want more sales. That’s...,1,2022,52,3,52,Between €500K-€ 1 million,27.0,NaN,NaN,"('https://www.linkedin.com/in/gerhardkuerner/',)",1
4,SurveySensum,1,https://www.eu-startups.com/directory/surveyse...,1,https://www.eu-startups.com/wp-content/uploads...,1,https://www.surveysensum.com,1,Salzburg,9,Tyrol,24.0,"finance, financial services, real estate",2,Media & Entertainment,17,SurveySensum is a leading customer feedback pl...,1,2023,37,2,37,Between €2.5 million-5 million,25.0,NaN,NaN,('https://www.linkedin.com/company/finslice-eu...,1
5,Artypa,1,https://www.eu-startups.com/directory/artypa/,1,https://www.eu-startups.com/wp-content/uploads...,1,https://artypa.com,1,Wien,8,Salzburg,13.0,"information services, information technology, ...",2,AgTech/FoodTech,15,Traditional AI workflows often involve navigat...,1,2018,31,7,31,Between €1-€100K,24.0,NaN,NaN,"('https://www.linkedin.com/company/daiki-ai/',)",1
6,Evologic Technologies,1,https://www.eu-startups.com/directory/evologic...,1,https://www.eu-startups.com/wp-content/uploads...,1,http://www.evologic-technologies.com/,1,Klagenfurt,4,Vorarlberg,8.0,"ai content, ai productivity, ai generation",1,Mobility,13,Evologic serves innovative companies with reli...,1,2017,24,8,24,Above €25 million,14.0,NaN,NaN,('https://www.linkedin.com/company/98882864/ad...,1
7,tubics,1,https://www.eu-startups.com/directory/tubics/,1,https://www.eu-startups.com/wp-content/uploads...,1,http://www.tubics.com,1,vienna,3,Carinthia,7.0,"podcast preparation, podcast research, topic r...",1,Education,12,tubics is a Software-as-a-Service platform tha...,1,2024,18,1,18,Between €5 million-€10 million,13.0,NaN,NaN,('https://www.linkedin.com/company/19018800/ad...,1
8,Fretello,1,https://www.eu-startups.com/directory/fretello/,1,https://www.eu-startups.com/wp-content/uploads...,1,https://fretello.com,1,Klosterneuburg,3,Burgenland,4.0,"wedding & event planning, lifestyle, productiv...",1,Hardware,12,Step-by-Step Guitar Lessons for Beginners. Enj...,1,2015,15,10,15,Between €10 million-€25 million,2.0,NaN,NaN,"('https://www.linkedin.com/company/idcanopy',)",1
9,Playerhunter,1,https://www.eu-startups.com/directory/playerhu...,1,https://www.eu-startups.com/wp-content/uploads...,1,https://playerhunter.com/,1,Niederosterreich,3,NaN,NaN,"fitness, health & w

In [ ]:
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.width')
pd.reset_option('display.max_colwidth')

#### Save df for further processing

In [27]:
df_detailed_listings.reset_index(drop=True, inplace=True)

In [29]:
df_detailed_listings.to_csv('./data/eustartup_listings_modified.csv', index=False)